In [441]:
%matplotlib inline

Loaded backend module://matplotlib_inline.backend_inline version unknown.
Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [442]:
# https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial6/Transformers_and_MHAttention.html

In [443]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})

In [444]:
import logging
 
logging.basicConfig(filename = 'mem_with_transf_synth_train.log',
                    level = logging.DEBUG,
                    format = '%(asctime)s:%(levelname)s:%(name)s:%(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

In [445]:
from tqdm import tqdm

In [446]:
import torchmetrics

In [447]:
import torch
import numpy as np
import pandas as pd
import math

In [448]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

In [449]:
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split

In [450]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda


In [451]:
train_context_query = np.load('train_context_query.npy', allow_pickle=True)
val_context_query = np.load('val_context_query.npy', allow_pickle=True)
test_context_query = np.load('test_context_query.npy', allow_pickle=True)

In [452]:
df_train= pd.read_json('synthetic_train.json')
df_val = pd.read_json('synthetic_val.json')
df_test = pd.read_json('synthetic_test.json')

In [453]:
print("The size of the synthetic train dataset is {}".format(len(df_train)))
print("The size of the synthetic val dataset is {}".format(len(df_val)))
print("The size of the synthetic test dataset is {}".format(len(df_test)))

The size of the synthetic train dataset is 94919
The size of the synthetic val dataset is 40680
The size of the synthetic test dataset is 58115


# non-overlapping

In [454]:
df_all = pd.concat([df_train, df_val, df_test])
all_context_query = np.hstack([train_context_query, val_context_query, test_context_query])

In [455]:
df_all.head()

,index,seq_len,seq,rep_token_first_pos,query_token,target_val
44654,44654,25,[81 9 32 70 73 68 19 85 1 30 15 45 82 64 38 ...,12,82,1
85524,85524,45,[54 48 92 66 35 94 53 95 24 49 4 5 72 62 15 ...,20,85,1
9508,9508,7,[54 30 8 1 19 71 73],0,54,1
21160,21160,13,[68 5 84 25 39 12 35 15 97 31 64 88 38],8,97,1
193670,193670,99,[44 50 31 65 97 60 95 40 42 30 14 62 10 33 72 ...,62,93,1


In [456]:
from sklearn.model_selection import train_test_split

train_val_query, test_query = train_test_split(df_all["query_token"].unique().tolist(), test_size=0.3, random_state=2, shuffle=True)
train_query, val_query = train_test_split(train_val_query, test_size=0.3, random_state=2, shuffle=True)

In [457]:
train_ids = df_all["query_token"].isin(train_query)
val_ids = df_all["query_token"].isin(val_query)
test_ids = df_all["query_token"].isin(test_query)

new_df_train = df_all[train_ids]
new_df_val = df_all[val_ids]
new_df_test = df_all[test_ids]

new_context_query_train = all_context_query[train_ids]
new_context_query_val = all_context_query[val_ids]
new_context_query_test = all_context_query[test_ids]

In [458]:
print("The size of the synthetic train dataset is {}".format(len(df_train)))
print("The size of the synthetic val dataset is {}".format(len(df_val)))
print("The size of the synthetic test dataset is {}".format(len(df_test)))

The size of the synthetic train dataset is 94919
The size of the synthetic val dataset is 40680
The size of the synthetic test dataset is 58115


# common

In [459]:
INP_PATH = '/data/sherin/'
orth_vectors = np.load(
    INP_PATH + 'orthonormal_vectors_512.npy')

In [ ]:
# get the input representations

In [460]:
#orth_vectors = np.random.rand(512, 512)

In [461]:
def get_tensors(inp, Orth):
    C = np.zeros((len(inp), 99, 512))
    Q = np.zeros((len(inp), 512))
    for idx, c_q in enumerate(inp):
        #print(c_q.shape)
        Q[idx] = Orth[c_q[-1]]
        C[idx,99 - (len(c_q)-1):,:] = Orth[c_q[:-1]]
    return C, Q

# Non overlapping

In [462]:
#train_C, train_Q = get_tensors(new_context_query_train, orth_vectors)

In [463]:
#val_C, val_Q = get_tensors(new_context_query_val, orth_vectors)
#test_C, test_Q = get_tensors(new_context_query_test, orth_vectors)

In [464]:
#train_label = new_df_train['target_val'].values
#val_label = new_df_val['target_val'].values
#test_label = new_df_test['target_val'].values

In [465]:
#len(new_context_query_test)

58176

# directly load the dataset of non overlapping queries

In [533]:
def get_tensors(inp, query, Orth):
    C = np.zeros((len(inp), 99, 512))
    Q = np.zeros((len(query), 512))
    for idx, c in enumerate(inp):
        #print(c)
        #Q[idx] = Orth[c_q[-1]]
        C[idx,99 - (len(c)):,:] = Orth[c]
    for idx, q in enumerate(query):
        Q[idx] = Orth[q]
    return C, Q

In [534]:
df_train = pd.read_json("no_overlap_syn_train.json")
df_val = pd.read_json("no_overlap_syn_val.json")
df_test = pd.read_json("no_overlap_syn_test.json")

In [535]:
new_context_train = df_train['sequence'].values
new_context_val = df_val['sequence'].values
new_context_test = df_test['sequence'].values

new_query_train = df_train['query'].values
new_query_val = df_val['query'].values
new_query_test = df_test['query'].values

In [540]:
train_C, train_Q = get_tensors(new_context_train, new_query_train, orth_vectors)
val_C, val_Q = get_tensors(new_context_val, new_query_val, orth_vectors)
test_C, test_Q = get_tensors(new_context_test, new_query_test, orth_vectors)

In [542]:
train_label = df_train['label'].values
val_label = df_val['label'].values
test_label = df_test['label'].values

# overlapping

In [149]:
train_C, train_Q = get_tensors(train_context_query, orth_vectors)
val_C, val_Q = get_tensors(val_context_query, orth_vectors)
test_C, test_Q = get_tensors(test_context_query, orth_vectors)

In [150]:
train_label = df_train['target_val'].values
val_label = df_val['target_val'].values
test_label = df_test['target_val'].values

# common

In [543]:
def get_data_loader(context_reps, query_reps, label, batch_size, shuffle):
    data_set = TensorDataset(context_reps, query_reps, label)
    loader = DataLoader(data_set, batch_size=batch_size, shuffle=shuffle)
    return loader

In [544]:
batch_size = 128

In [545]:
train_loader = get_data_loader(torch.tensor(train_C).float(), torch.tensor(train_Q).float(),
                               torch.tensor(train_label), batch_size, shuffle=True)
val_loader = get_data_loader(torch.tensor(val_C).float(), torch.tensor(val_Q).float(),
                             torch.tensor(val_label), batch_size, shuffle=False)
test_loader = get_data_loader(torch.tensor(test_C).float(), torch.tensor(test_Q).float(),
                              torch.tensor(test_label), batch_size, shuffle=False)

In [594]:
class LSTMPredictor(nn.Module):

    def __init__(self, input_dim=512, model_dim=32,
                 num_heads=1, num_layers=1,
                 dropout=0.0, input_dropout=0.0):

        super().__init__()

        # Input dim -> Model dim
        self.input_net = nn.Sequential(
            #nn.Dropout(input_dropout),
            nn.Linear(input_dim, model_dim)
        )
        self.output_net = nn.Sequential(
            #nn.Dropout(input_dropout),
            nn.Linear(128, input_dim)
        )
        # LSTM
        self.lstm = nn.LSTM(512, 128, batch_first=True)

    def forward(self, x, y):
        #x = self.input_net(x)
        #y = self.input_net(y)
        #print(x.shape)
        #print(y.shape)
        #x = self.positional_encoding(x)
        #y = self.positional_encoding(y.unsqueeze(1))
        #h_0 = Variable(torch.randn(1, 128, 240).cuda())
        #c_0 = Variable(torch.randn(1, 128, 240).cuda())
        
        #print(x.shape)
        
        lstm_out, (hidden_state, cell_state) = self.lstm(x)
        #x = torch.hstack([hidden_state[0, :, :], cell_state[0, :, :]])
        x =hidden_state[0, :, :]
        #x = self.lstm(x)[0]
        x = self.output_net(x)

        #op = torch.sum(x*y, dim=1)
        #print(op.shape)
        op = torch.sum(x*y, dim=1)
        return op

In [606]:
model = LSTMPredictor(model_dim=240).to(device)

In [607]:
model

LSTMPredictor(
  (input_net): Sequential(
    (0): Linear(in_features=512, out_features=240, bias=True)
  )
  (output_net): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
  )
  (lstm): LSTM(512, 128, batch_first=True)
)

In [608]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

In [609]:
print(params)

517872


In [610]:
#checkpoint_path = '/data/sherin/checkpoint_lm/chkpt_synth_tran_posenc_steplr_recall_best.pt.tar'
checkpoint_path = '/data/sherin/checkpoint_lm/chkpt_synth_lstm_dim_240_steplr_recall_best.pt.tar'

In [614]:
num_epochs = 100
criterion = nn.BCEWithLogitsLoss()
#weight_decay=1e-5 -> l2 regularization; weight decay very small means you are not penalizing it; 
# weight decay large means you are penalizing weights.
#optimizer = optim.RMSprop(model.parameters(), lr=1e-3, weight_decay=1e-5)
optimizer = optim.RMSprop(model.parameters(), lr=1e-3, weight_decay=1e-4)

#optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
#optimizer = optim.RMSprop(model.parameters(), lr=0.0013378606854350151)
#scheduler = StepLR(optimizer, step_size=5, gamma=0.8)
#scheduler = CosineWarmupScheduler(optimizer, warmup=50,
                                             #max_iters=num_epochs*len(train_loader))

In [615]:
from torch.autograd import Variable

In [616]:
epoch_loss_list = []
accuracy_list = []
val_loss_list = []
val_acc_list = []
valid_acc_max = 0 

for epoch in range(num_epochs):
    train_count = 0
    model.train()
    epoch_loss = 0.0
    accuracy = 0.0
    

    for context, query, labels in tqdm(train_loader):
    #for context, query, labels in tqdm(val_loader):
        train_count = train_count+1
        context = context.to(device)
        query = query.to(device)    
        target = labels.to(device)
        label = labels.float().to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(context, query)
        loss = criterion(outputs, label)
        loss.backward()

        optimizer.step()
        
        epoch_loss += loss.item()
        accuracy += torchmetrics.functional.accuracy(outputs, target, threshold=0.5).item()
        
        #scheduler.step()

    
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    
    test_count = 0
    for context, query, labels in tqdm(val_loader):
        test_count = test_count + 1
        context = context.to(device)
        query = query.to(device)
        target = labels.to(device)
        label = labels.float().to(device)
        
        outputs = model(context, query)
        loss = criterion(outputs, label)
        val_loss += loss.item()
        val_acc += torchmetrics.functional.accuracy(outputs, target, threshold=0.5).item()
        
    accuracy = accuracy / train_count
    epoch_loss = epoch_loss / train_count
    val_loss = val_loss / test_count
    val_acc = val_acc / test_count
    
    if val_acc > valid_acc_max:
        logging.info("saving best model")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
            'accuracy': val_acc,
            }, checkpoint_path)
        valid_acc_max = val_acc
    else:
        logging.info("not saving the model")
    
    curr_lr = optimizer.param_groups[0]['lr']
    logging.info(f'curr_lr: {curr_lr}')
    logging.info(f'[{epoch + 1}] Training loss: {epoch_loss:.3f} Training accuracy : {accuracy:.3f}')
    logging.info(f'[{epoch + 1}] Validation loss: {val_loss:.3f} Validation accuracy : {val_acc:.3f}')
    epoch_loss_list.append(epoch_loss)
    accuracy_list.append(accuracy)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)
    
    # scheduler.step()

print('Finished Training')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:02<00:00, 57.57it/s]
saving best model
saving best model
saving best model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[1] Training loss: 0.671 Training accuracy : 0.538
[1] Training loss: 0.671 Training accuracy : 0.538
[1] Training loss: 0.671 Training accuracy : 0.538
[1] Validation loss: 0.694 Validation accuracy : 0.503
[1] Validation loss: 0.694 Validation accuracy : 0.503
[1] Validation loss: 0.694 Validation accuracy : 0.503
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:01<00:00, 64.24it/s]
not saving the model
not saving the model
not saving the model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[2] Training loss: 0.662 Training accuracy : 0.552
[2] Training loss: 0.6

[10] Validation loss: 0.693 Validation accuracy : 0.503
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:01<00:00, 63.05it/s]
not saving the model
not saving the model
not saving the model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[11] Training loss: 0.628 Training accuracy : 0.614
[11] Training loss: 0.628 Training accuracy : 0.614
[11] Training loss: 0.628 Training accuracy : 0.614
[11] Validation loss: 0.693 Validation accuracy : 0.503
[11] Validation loss: 0.693 Validation accuracy : 0.503
[11] Validation loss: 0.693 Validation accuracy : 0.503
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:02<00:00, 58.70it/s]
not saving the model
not saving the model
not saving the model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[1

[20] Validation loss: 0.693 Validation accuracy : 0.503
[20] Validation loss: 0.693 Validation accuracy : 0.503
[20] Validation loss: 0.693 Validation accuracy : 0.503
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:02<00:00, 60.03it/s]
not saving the model
not saving the model
not saving the model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[21] Training loss: 0.579 Training accuracy : 0.671
[21] Training loss: 0.579 Training accuracy : 0.671
[21] Training loss: 0.579 Training accuracy : 0.671
[21] Validation loss: 0.693 Validation accuracy : 0.503
[21] Validation loss: 0.693 Validation accuracy : 0.503
[21] Validation loss: 0.693 Validation accuracy : 0.503
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:02<00:00, 59.49it/s

[30] Training loss: 0.518 Training accuracy : 0.725
[30] Training loss: 0.518 Training accuracy : 0.725
[30] Validation loss: 0.693 Validation accuracy : 0.503
[30] Validation loss: 0.693 Validation accuracy : 0.503
[30] Validation loss: 0.693 Validation accuracy : 0.503
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:02<00:00, 58.71it/s]
not saving the model
not saving the model
not saving the model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[31] Training loss: 0.510 Training accuracy : 0.731
[31] Training loss: 0.510 Training accuracy : 0.731
[31] Training loss: 0.510 Training accuracy : 0.731
[31] Validation loss: 0.693 Validation accuracy : 0.503
[31] Validation loss: 0.693 Validation accuracy : 0.503
[31] Validation loss: 0.693 Validation accuracy : 0.503
100%|██████████████████████████████████████████████████████████████████████████████████████████

not saving the model
not saving the model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[40] Training loss: 0.462 Training accuracy : 0.767
[40] Training loss: 0.462 Training accuracy : 0.767
[40] Training loss: 0.462 Training accuracy : 0.767
[40] Validation loss: 0.693 Validation accuracy : 0.503
[40] Validation loss: 0.693 Validation accuracy : 0.503
[40] Validation loss: 0.693 Validation accuracy : 0.503
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:01<00:00, 61.95it/s]
not saving the model
not saving the model
not saving the model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[41] Training loss: 0.459 Training accuracy : 0.767
[41] Training loss: 0.459 Training accuracy : 0.767
[41] Training loss: 0.459 Training accuracy : 0.767
[41] Validation loss: 0.693 Validation accuracy : 0.503
[41] Validation loss: 0.693 Validation accuracy : 0.503
[41] Validat

...

[69] Validation loss: 0.693 Validation accuracy : 0.503
[69] Validation loss: 0.693 Validation accuracy : 0.503
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:01<00:00, 61.95it/s]
not saving the model
not saving the model
not saving the model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[70] Training loss: 0.406 Training accuracy : 0.803
[70] Training loss: 0.406 Training accuracy : 0.803
[70] Training loss: 0.406 Training accuracy : 0.803
[70] Validation loss: 0.693 Validation accuracy : 0.503
[70] Validation loss: 0.693 Validation accuracy : 0.503
[70] Validation loss: 0.693 Validation accuracy : 0.503
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:01<00:00, 70.45it/s]
not saving the model
not saving the model
not saving t

[79] Training loss: 0.402 Training accuracy : 0.807
[79] Validation loss: 0.693 Validation accuracy : 0.503
[79] Validation loss: 0.693 Validation accuracy : 0.503
[79] Validation loss: 0.693 Validation accuracy : 0.503
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:02<00:00, 60.83it/s]
not saving the model
not saving the model
not saving the model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[80] Training loss: 0.400 Training accuracy : 0.808
[80] Training loss: 0.400 Training accuracy : 0.808
[80] Training loss: 0.400 Training accuracy : 0.808
[80] Validation loss: 0.693 Validation accuracy : 0.503
[80] Validation loss: 0.693 Validation accuracy : 0.503
[80] Validation loss: 0.693 Validation accuracy : 0.503
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

curr_lr: 0.001
curr_lr: 0.001
[89] Training loss: 0.398 Training accuracy : 0.810
[89] Training loss: 0.398 Training accuracy : 0.810
[89] Training loss: 0.398 Training accuracy : 0.810
[89] Validation loss: 0.693 Validation accuracy : 0.503
[89] Validation loss: 0.693 Validation accuracy : 0.503
[89] Validation loss: 0.693 Validation accuracy : 0.503
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:01<00:00, 83.51it/s]
not saving the model
not saving the model
not saving the model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[90] Training loss: 0.397 Training accuracy : 0.810
[90] Training loss: 0.397 Training accuracy : 0.810
[90] Training loss: 0.397 Training accuracy : 0.810
[90] Validation loss: 0.693 Validation accuracy : 0.503
[90] Validation loss: 0.693 Validation accuracy : 0.503
[90] Validation loss: 0.693 Validation accuracy : 0.503
100%|████████

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:01<00:00, 64.09it/s]
not saving the model
not saving the model
not saving the model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[99] Training loss: 0.394 Training accuracy : 0.812
[99] Training loss: 0.394 Training accuracy : 0.812
[99] Training loss: 0.394 Training accuracy : 0.812
[99] Validation loss: 0.693 Validation accuracy : 0.503
[99] Validation loss: 0.693 Validation accuracy : 0.503
[99] Validation loss: 0.693 Validation accuracy : 0.503
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 123/123 [00:01<00:00, 68.09it/s]
not saving the model
not saving the model
not saving the model
curr_lr: 0.001
curr_lr: 0.001
curr_lr: 0.001
[100] Training loss: 0.393 Training accuracy : 0.812
[100]

Finished Training


In [ ]:
print("finished")

In [252]:
PATH = checkpoint_path
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
acc = checkpoint['accuracy']

# inferece
model.eval()

TransformerPredictor(
  (input_net): Sequential(
    (0): Linear(in_features=512, out_features=240, bias=True)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): EncoderBlock(
        (self_attn): MultiheadAttention(
          (qkv_proj): Linear(in_features=240, out_features=720, bias=True)
          (o_proj): Linear(in_features=240, out_features=240, bias=True)
        )
        (linear_net): Sequential(
          (0): Linear(in_features=240, out_features=480, bias=True)
          (1): Dropout(p=0.0, inplace=False)
          (2): ReLU(inplace=True)
          (3): Linear(in_features=480, out_features=240, bias=True)
        )
        (norm1): LayerNorm((240,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((240,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (1): EncoderBlock(
        (self_attn): MultiheadAttention(
          (qkv_proj): Linear(in_features=240, out_features=720, bias=True)

In [253]:
print(acc)

0.9991892688679245


In [254]:
test_count = 0
output_logits_trans = []
test_acc = 0
for context, query, labels in tqdm(test_loader):
    test_count = test_count + 1
    context = context.to(device)
    query = query.to(device)
    target = labels.to(device)
    label = labels.float().to(device)

    outputs = model(context, query)
    output_logits = outputs.detach().cpu().numpy()
    test_acc += torchmetrics.functional.accuracy(outputs, target, threshold=0.5).item()
    output_logits_trans.append(output_logits)

accuracy = test_acc/test_count
print("The test accuracy is {}".format(accuracy))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [00:25<00:00, 17.72it/s]

The test accuracy is 0.9988839285714286


In [165]:
test_output_logits_trans = np.hstack(output_logits_trans)

In [166]:
trans_pred = torch.sigmoid(torch.tensor(test_output_logits_trans))
trans_pred_label = 1.0 * (trans_pred > 0.5)

In [167]:
torch.save(trans_pred, 'new_trans_pred.pt')
torch.save(trans_pred_label, 'new_trans_pred_label.pt')